In [39]:
from enum import Enum
from typing import List, Tuple
from collections import Counter, OrderedDict
from itertools import combinations, combinations_with_replacement

In [2]:
DICE_PER_ROLL = 5
DICE_VALUES = [1,2,3,4,5,6]

In [3]:
def sum_of_single_value(input_list: Tuple[int], value: int):
    return input_list.count(value) * value

In [4]:
def n_of_a_kind(input_list: Tuple[int], n: int):
    for value, count in Counter(input_list).items():
        if count >= n:
            if n == DICE_PER_ROLL:
                return 50
            else:
                return sum(input_list)
    return 0

In [5]:
def full_house(input_list: Tuple[int]):
    if set(Counter(input_list).values()) == {3, 2}:
        return 25
    else:
        return 0

In [6]:
def find_longest_sequence(input_list: Tuple[int]):
    max_seq = 1
    nums = set(input_list)
    for num in nums:
        this_seq = 1
        seq = True
        while seq:
            if num + this_seq in nums:
                this_seq += 1
            else:
                seq = False
        
        max_seq = max(this_seq, max_seq)
    
    return max_seq

In [7]:
def straight(input_list: Tuple[int], size: int):
    seq = find_longest_sequence(input_list)
    
    if seq >= size:
        if size == DICE_PER_ROLL:
            return 40
        elif size == DICE_PER_ROLL - 1:
            return 30
        
    return 0

In [97]:
hand_scoring_functions = [
    lambda x: sum_of_single_value(x, 1),
    lambda x: sum_of_single_value(x, 2),
    lambda x: sum_of_single_value(x, 3),
    lambda x: sum_of_single_value(x, 4),
    lambda x: sum_of_single_value(x, 5),
    lambda x: sum_of_single_value(x, 6),
    lambda x: n_of_a_kind(x, 3),
    lambda x: n_of_a_kind(x, 4),
    lambda x: full_house(x),
    lambda x: straight(x, 4),
    lambda x: straight(x, 5),
    lambda x: n_of_a_kind(x, 5),
    lambda x: sum(x)
]

In [98]:
def calculate_all_scores(input_list: Tuple[int]):
    return [func(input_list) for func in hand_scoring_functions]

In [99]:
def calculate_maximum_score(input_list: Tuple[int]):
    return max(calculate_all_scores(input_list))

In [100]:
def calculate_possible_final_rolls(saved_dice: Tuple[int]):
    num_to_roll = DICE_PER_ROLL - len(saved_dice)
    return [tuple(sorted(saved_dice + combo)) for combo in 
            combinations_with_replacement(DICE_VALUES, num_to_roll)]

In [101]:
def calculate_expected_score(saved_dice: Tuple[int]):
    final_rolls = calculate_possible_final_rolls(saved_dice)
    running_expected_score = 0
    for roll in final_rolls:
        running_expected_score += calculate_maximum_score(roll) / len(final_rolls)
        
    return running_expected_score

In [102]:
def calculate_all_save_possibilities(input_list: Tuple[int]):
    saved_dice = {tuple(), }
    for i in range(1, len(input_list)+1):
        for combo in combinations(input_list, i):
            saved_dice.add(tuple(sorted(combo)))
            
    return saved_dice

In [103]:
def simulate(second_to_last_roll: Tuple[int]):
    result = {possible_saved_dice: calculate_expected_score(possible_saved_dice) 
                        for possible_saved_dice in calculate_all_save_possibilities(second_to_last_roll)}
    
    print(f'{"saved_dice":<18}score')
    for (saved_roll, expected) in sorted(result.items(), key=lambda x: x[1]):
        print(f'{str(saved_roll):<18}{expected:.2f}')

In [114]:
simulate((1,4,5,4,5))

saved_dice        score
(1,)              17.30
(1, 5)            17.50
(1, 4, 4, 5)      17.50
(1, 4, 4)         17.95
(1, 4)            18.09
(1, 4, 5, 5)      18.50
(1, 4, 5)         18.71
(1, 5, 5)         18.95
(1, 4, 4, 5, 5)   19.00
()                20.06
(4,)              20.51
(5,)              20.67
(4, 4)            20.86
(4, 4, 5)         21.19
(4, 5)            22.02
(4, 5, 5)         22.10
(5, 5)            22.16
(4, 4, 5, 5)      22.33


In [110]:
calculate_all_scores((1,1,1,2,3))

[3, 2, 3, 0, 0, 0, 8, 0, 0, 0, 0, 0, 8]